In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_property2_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20'].drop(['arnold_fta_raw', 'arnold_nca_raw', 'arnold_nvca_raw'], axis=1)
y = data['property_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:



In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.7877614464717588, 0.00830832594070947)

In [6]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.7938857509964382, 0.005500793596808151)

### Lasso Stumps

In [7]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')

## split X & Y
X_stumps = data.loc[:,:'current_violence201'].copy()
Y_stumps = data['property_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_train_stumps.csv").sort_values('person_id')
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test_stumps.csv").sort_values('person_id')
X_train_stumps = train_stumps.loc[:,:'current_violence201'].copy()
X_test_stumps = test_stumps.loc[:,:'current_violence201'].copy()
Y_train_stumps = train_stumps['property_two_year'].values.copy()
Y_test_stumps = test_stumps['property_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [8]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.002, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

26

In [9]:
single_stump_model['features']

['sex1',
 'age_at_current_charge20',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge35',
 'age_at_current_charge39',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_felony1',
 'p_felony2',
 'p_felony4',
 'p_misdemeanor1',
 'p_misdemeanor3',
 'p_property1',
 'p_property2',
 'p_drug1',
 'ADE1',
 'p_fta_two_year1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'current_pending_charge1']

#### Nested Cross Validation

In [10]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [11]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.7927124720493229,
 0.000852903456425702)

### RiskSLIM

In [15]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.0008,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

17

In [16]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [17]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='property_two_year', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 01:54 PM | 223 rows in lookup table
02/12/20 @ 01:54 PM | ------------------------------------------------------------
02/12/20 @ 01:54 PM | runnning initialization procedure
02/12/20 @ 01:54 PM | ------------------------------------------------------------
02/12/20 @ 01:54 PM | CPA produced 2 cuts
02/12/20 @ 01:54 PM | running naive rounding on 6 solutions
02/12/20 @ 01:54 PM | best objective value: 0.5235
02/12/20 @ 01:54 PM | rounding produced 5 integer solutions
02/12/20 @ 01:54 PM | best objective value is 0.5113
02/12/20 @ 01:54 PM | running sequential rounding on 6 solutions
02/12/20 @ 01:54 PM | best objective value: 0.5235
02/12/20 @ 01:54 PM | sequential rounding produced 3 integer solutions
02/12/20 @ 01:54 PM | best objective value: 0.3279
02/12/20 @ 01:54 PM | polishing 8 solutions
02/12/20 @ 01:54 PM | best objective value: 0.3279
02/12/20 @ 01:54 PM | polishing produced 4 integer solutions
02/12/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 01:54 PM | adding 385 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1575.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Elapsed time = 48.00 sec. (41271.62 ticks, tree = 7.32 MB, solutions = 6)
  87199 23841        0.1491    14        0.1520        0.1471   665550    3.23%          rho_11 D  87199  87191     32
  88627 24313        cutoff              0.1520        0.1471   677667    3.23%          rho_13 D  88627  88560     35
  90036 24836        0.1474    15        0.1520        0.1471   689013    3.23%          rho_10 D  90036  90035     25
  91395 25263        0.1471     9        0.1520        0.1471   700353    3.23%        alpha_17 U  91395  91387     32
  92768 25686        0.1520    13        0.1520        0.1471   711677    3.23%          rho_10 N  92768  92766     27
  94141 26132        cutoff              0.1520        0.1471   723265    3.23%           rho_1 D  94141  94139     29
* 95477 25767      integral     0        0.1517        0.1471   733544    3.02%
  95502 25772        0.1492    10        0.1517        0.1471   733677    3.02%          rho_13 U  95502  95501     34
* 96049 25960

02/12/20 @ 01:57 PM | best objective value: 0.5252
02/12/20 @ 01:57 PM | rounding produced 5 integer solutions
02/12/20 @ 01:57 PM | best objective value is 0.5115
02/12/20 @ 01:57 PM | running sequential rounding on 5 solutions
02/12/20 @ 01:57 PM | best objective value: 0.5252
02/12/20 @ 01:57 PM | sequential rounding produced 3 integer solutions
02/12/20 @ 01:57 PM | best objective value: 0.3271
02/12/20 @ 01:57 PM | polishing 8 solutions
02/12/20 @ 01:57 PM | best objective value: 0.3271
02/12/20 @ 01:57 PM | polishing produced 4 integer solutions
02/12/20 @ 01:57 PM | initialization produced 12 feasible solutions
02/12/20 @ 01:57 PM | best objective value: 0.1569
02/12/20 @ 01:57 PM | ------------------------------------------------------------
02/12/20 @ 01:57 PM | completed initialization procedure
02/12/20 @ 01:57 PM | ------------------------------------------------------------
02/12/20 @ 01:57 PM | 223 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CP

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 01:57 PM | adding 348 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1569.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

  83799 26819        0.1478    12        0.1515        0.1466   651903    3.23%           rho_3 U  83799  83798     28
  85244 27074        0.1472    15        0.1515        0.1467   660804    3.19%           rho_5 D  85244  82242     31
  86635 27261        0.1490    11        0.1515        0.1467   669892    3.15%          rho_13 D  86635  75061     30
  88069 27479        cutoff              0.1515        0.1468   679031    3.12%        alpha_16 U  88069  88068     32
Elapsed time = 25.88 sec. (41265.86 ticks, tree = 8.47 MB, solutions = 11)
  89475 27595        0.1505    15        0.1515        0.1468   687806    3.08%           rho_9 N  89475  89474     18
  90847 27757        0.1505     8        0.1515        0.1469   696647    3.04%        alpha_14 D  90847  90846     34
  92234 27848        cutoff              0.1515        0.1469   705323    3.01%           rho_0 U  92234   9276     15
  93593 27955        0.1473    16        0.1515        0.1470   714056    2.97%           rh

02/12/20 @ 02:40 PM | ------------------------------------------------------------
02/12/20 @ 02:40 PM | runnning initialization procedure
02/12/20 @ 02:40 PM | ------------------------------------------------------------
02/12/20 @ 02:40 PM | CPA produced 2 cuts
02/12/20 @ 02:40 PM | running naive rounding on 5 solutions
02/12/20 @ 02:40 PM | best objective value: 0.5304
02/12/20 @ 02:40 PM | rounding produced 5 integer solutions
02/12/20 @ 02:40 PM | best objective value is 0.5125
02/12/20 @ 02:40 PM | running sequential rounding on 5 solutions
02/12/20 @ 02:40 PM | best objective value: 0.5304
02/12/20 @ 02:40 PM | sequential rounding produced 3 integer solutions
02/12/20 @ 02:40 PM | best objective value: 0.3282
02/12/20 @ 02:40 PM | polishing 8 solutions
02/12/20 @ 02:40 PM | best objective value: 0.3282
02/12/20 @ 02:40 PM | polishing produced 4 integer solutions
02/12/20 @ 02:40 PM | initialization produced 12 feasible solutions
02/12/20 @ 02:40 PM | best objective value: 0.1597

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 02:40 PM | adding 325 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1597.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  76171 23524        0.1508    14        0.1519        0.1475   612599    2.88%           rho_0 U  76171  13030     28
  77839 23777        cutoff              0.1519        0.1476   623736    2.84%          rho_10 N  77839  77838     20
  79515 24055        0.1502    15        0.1519        0.1476   634297    2.80%           rho_5 D  79515  79514     22
  81193 24347        0.1487    16        0.1519        0.1477   645143    2.77%          rho_10 U  81193  46658     24
Elapsed time = 25.17 sec. (41266.69 ticks, tree = 7.67 MB, solutions = 11)
  82834 24580        cutoff              0.1519        0.1477   655559    2.74%          rho_12 D  82834  72578     23
  84452 24771        0.1506    12        0.1519        0.1478   666292    2.70%          rho_16 D  84452  84451     33
  86090 24961        0.1506    13        0.1519        0.1478   676510    2.66%           rho_4 N  86090  35866     27
  87726 25125        0.1511    10        0.1519        0.1479   686822    2.63%          rho

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning:

invalid value encountered in longlong_scalars



In [18]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.7589717158388861, 0.7594714973326554)

#### Single RiskSLIM Model

In [19]:
selected_features = ["property_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y+
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [20]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'property_two_year',
    'sample_weights': sample_weights
}

In [21]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
02/12/20 @ 02:46 PM | 223 rows in lookup table
02/12/20 @ 02:46 PM | ------------------------------------------------------------
02/12/20 @ 02:46 PM | runnning initialization procedure
02/12/20 @ 02:46 PM | ------------------------------------------------------------
02/12/20 @ 02:46 PM | CPA produced 2 cuts
02/12/20 @ 02:46 PM | running naive rounding on 5 solutions
02/12/20 @ 02:46 PM | best objective value: 0.5215
02/12/20 @ 02:46 PM | rounding produced 5 integer solutions
02/12/20 @ 02:46 PM | best objective value is 0.5120
02/12/20 @ 02:46 PM | running sequential rounding on 5 solutions
02/12/20 @ 02:46 PM | best objective value: 0.5215
02/12/20 @ 02:46 PM | sequential rounding produced 3 integer solutions
02/12/20 @ 02:46 PM | best objective value: 0.3279
02/12/20 @ 02:46 PM | polishing 8 solutions
02/12/20 @ 02:46 PM | best objective value: 0.3279
02/12/20 @ 02:46 PM | polishing produced 4 integer solutions
02/12/20 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/12/20 @ 02:46 PM | adding 347 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1570.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  76235 20210        cutoff              0.1513        0.1469   653643    2.92%           rho_5 N  76235  20277     37
Elapsed time = 32.22 sec. (41269.50 ticks, tree = 6.60 MB, solutions = 7)
  77724 20369        cutoff              0.1513        0.1470   663624    2.88%          rho_15 U  77724  77722     21
  79200 20538        cutoff              0.1513        0.1470   673393    2.85%           rho_7 U  79200  79199     35
  80645 20627        0.1508    14        0.1513        0.1471   683235    2.81%           rho_8 N  80645  18089     22
  82103 20720        0.1503    12        0.1513        0.1471   693072    2.77%           rho_4 U  82103  82102     35
  83537 20820        cutoff              0.1513        0.1472   702918    2.73%           rho_6 U  83537  83535     23
  85008 20912        cutoff              0.1513        0.1472   712490    2.70%          rho_17 D  85008  85006     32
  86482 20987        0.1507    12        0.1513        0.1473   721730    2.66%           rho

In [22]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [23]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7625854990471765

In [24]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7497432422652096

### Arnold PSA

In [28]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['property_two_year'].values

In [29]:
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')

In [30]:
np.mean(arnold['auc'])

0.7248307038595256

#### Single Arnold PSA

In [31]:
### load data
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_test.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    test_data = test_data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    test_data[score] = test_data[score + "_raw"]
    test_data = test_data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        test_data[score] = test_data[score].astype(int)

In [32]:
X = test_data['arnold_nca'].values
Y = test_data['property_two_year'].values
roc_auc_score(Y, X)

0.7219548204367592

### Results

In [33]:
#### save results
summary_property2_KY_interpret = {"cart": cart_summary,
                                  "ebm": ebm_summary, 
                                  'stumps': stump_summary, 
                                  'riskslim': riskslim_summary, 
                                  'arnold': arnold}
%store summary_property2_KY_interpret

Stored 'summary_property2_KY_interpret' (dict)


In [34]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])]]
results

[['cart', 0.7877614464717588, 0.00830832594070947],
 ['ebm', 0.7938857509964382, 0.005500793596808151],
 ['stumps', 0.7927124720493229, 0.000852903456425702],
 ['riskslim', 0.7594714973326554],
 ['arnold', 0.7248307038595256]]

In [35]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [36]:
path = "./kentucky/logs/interpretable/"
results = [["Property", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")"]]
with open(path + 'KY-two-year-interpretable-summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Save Fairness Matrix

In [37]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [38]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './kentucky/fairness/' + name[i] + '/two-year/property/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './kentucky/fairness/' + name[i] + '/two-year/property/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)